In [1]:
import pandas as pd
import numpy as np
from math import exp
from utils import get_names2abbrs_dict
from utils import get_predictions_df

In [2]:
raw = True
start_date = '2020-05-01'
end_date = '2020-10-31'

In [3]:
state_po_votes = pd.read_csv('label_data/electoral_votes_2020.csv')

In [4]:
d_probs = pd.read_csv(f'poll_data/all_pres_polls_d_probs{"_raw" if raw else ""}.csv')

In [5]:
kwargs = {'start_date': start_date, 'end_date': end_date, 'alpha': 0.4, 'logistic': True}
df = get_predictions_df(d_probs, **kwargs)
filename = f'predictions/presidential_{"raw_" if raw else ""}preds_{"logistic" if kwargs["logistic"] else "lambda"}_{"-".join(str(kwargs["alpha"]).split("."))}_{start_date}-{end_date}.csv'
df.to_csv(filename, index=False)
print(f'wrote to {filename}')

wrote to predictions/presidential_raw_preds_logistic_0-4_2020-05-01-2020-10-31.csv


In [6]:
kwargs = {'start_date': start_date, 'end_date': end_date, 'alpha': 0.2, 'logistic': False}
df = get_predictions_df(d_probs, **kwargs)
filename = f'predictions/presidential_{"raw_" if raw else ""}preds_{"logistic" if kwargs["logistic"] else "lambda"}_{"-".join(str(kwargs["alpha"]).split("."))}_{start_date}-{end_date}.csv'
df.to_csv(filename, index=False)
print(f'wrote to {filename}')

wrote to predictions/presidential_raw_preds_lambda_0-2_2020-05-01-2020-10-31.csv


In [7]:
kwargs = {'start_date': start_date, 'end_date': end_date, 'alpha': 1, 'logistic': False}
df = get_predictions_df(d_probs, **kwargs)
filename = f'predictions/presidential_{"raw_" if raw else ""}preds_{"logistic" if kwargs["logistic"] else "lambda"}_{"-".join(str(kwargs["alpha"]).split("."))}_{start_date}-{end_date}.csv'
df.to_csv(filename, index=False)
print(f'wrote to {filename}')

wrote to predictions/presidential_raw_preds_lambda_1_2020-05-01-2020-10-31.csv
